#REAL-TIME FABRIC CLASSIFICATION USING **ONNX** AND **COMPUTER VISION**
🧪 CLOTHING MATERIAL CLASSIFICATION – TESTING MODULE :
  Use this after model training (cloth_model_mobilenetv2.h5)

***Author: Vinodha.s***



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import cv2
from PIL import Image

# Paths and constants
DATASET_DIR = r"C:\Users\vinod\Downloads\project\dataset_split"  # Base dir with 'train' and 'val' subfolders
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
MODEL_SAVE_PATH = "cloth_model_mobilenetv2.h5"

import os

# Define dataset folder paths
train_dir = r"C:\Users\vinod\Downloads\project\dataset_split\train"
val_dir = r"C:\Users\vinod\Downloads\project\dataset_split\validation"

print("Train folder exists:", os.path.exists(train_dir))
print("Validation folder exists:", os.path.exists(val_dir))

# Step 2: Check consistency
train_classes = sorted(os.listdir(train_dir))
val_classes = sorted(os.listdir(val_dir))

print("\nTrain classes:", train_classes)
print("Validation classes:", val_classes)

missing_in_train = [cls for cls in val_classes if cls not in train_classes]
missing_in_val = [cls for cls in train_classes if cls not in val_classes]

if missing_in_train:
    print("⚠️ Found in validation but missing in train:", missing_in_train)
if missing_in_val:
    print("⚠️ Found in train but missing in validation:", missing_in_val)
if not missing_in_train and not missing_in_val:
    print("✅ Both train and validation have same class folders.")


# Define train and val directories
#train_dir = os.path.join(DATASET_DIR, 'train')
#val_dir = os.path.join(DATASET_DIR, 'val')

# Check if directories exist
if not os.path.exists(train_dir) or not os.path.exists(val_dir):
    raise FileNotFoundError(f"Dataset directories not found. Ensure {train_dir} and {val_dir} exist and contain class subfolders with images.")

# Data augmentation (no validation_split needed since data is pre-split)
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Generators pointing directly to train and val folders
train_gen = datagen.flow_from_directory(
    train_dir,  # Points to 'train' folder
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    val_dir,  # Points to 'val' folder
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("Train samples:", train_gen.samples)
print("Validation samples:", val_gen.samples)
print("Classes:", train_gen.class_indices)


# Label mapping (should now show fabric classes like 'Cotton', 'Silk', etc.)
label_map = {v: k for k, v in train_gen.class_indices.items()}
print("Label map:", label_map)

# Model building
base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Callbacks
checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)



# Training
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stopping]
)

# Evaluation
loss, acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {acc * 100:.2f}%")

# Pie chart for accuracy
plt.figure(figsize=(6, 6))
plt.pie(
    [acc * 100, 100 - acc * 100],
    labels=['Correct', 'Incorrect'],
    autopct='%1.1f%%',
    startangle=90,
    colors=['#4CAF50', '#E57373']
)
plt.title("Model Accuracy on Validation Data")
plt.show()

# Fabric info dictionary
FABRIC_INFO = {
    "Cotton": {"mix": "Pure (Natural)", "bio": "Yes", "time": "5-6 months", "recycle": "Yes"},
    "Silk": {"mix": "Pure (Natural)", "bio": "Yes", "time": "4-5 months", "recycle": "Limited"},
    "Wool": {"mix": "Pure (Natural)", "bio": "Yes", "time": "1-5 months", "recycle": "Yes"},
    "Linen": {"mix": "Pure (Natural)", "bio": "Yes", "time": "2-4 months", "recycle": "Yes"},
    "Denim": {"mix": "Cotton + Elastane", "bio": "Partial", "time": "10-12 months", "recycle": "Yes"},
    "Polyester": {"mix": "Pure (Synthetic)", "bio": "No", "time": "20-200 years", "recycle": "Yes"},
    "Nylon": {"mix": "Pure (Synthetic)", "bio": "No", "time": "30-40 years", "recycle": "Yes"},
    "Rayon": {"mix": "Semi-Synthetic", "bio": "Partial", "time": "1-2 years", "recycle": "Limited"},
    "Leather": {"mix": "Processed Animal Hide", "bio": "No", "time": "10-50 years", "recycle": "Hard"},
    "Velvet": {"mix": "Blended", "bio": "Depends", "time": "Varies", "recycle": "Depends"}
}

# Function to predict on a single image
def predict_image(model, img_path):
    if not os.path.exists(img_path):
        print(f"Error: Image path {img_path} does not exist.")
        return

    img = Image.open(img_path).convert('RGB')
    img = img.resize(IMG_SIZE)
    arr = np.expand_dims(np.array(img) / 255.0, axis=0)
    pred = model.predict(arr)
    idx = np.argmax(pred)
    label = label_map[idx]
    conf = np.max(pred)
    info = FABRIC_INFO.get(label, {"mix": "-", "bio": "-", "time": "-", "recycle": "-"})

    print(f"\n🧵 Type of cloth: {label}")
    print(f"♦️ Mixing Fabric: {info['mix']}")
    print(f"🌱 Biodegradable: {info['bio']}")
    print(f"⌛ Decomposition Time: {info['time']}")
    print(f"♻️ Recyclable: {info['recycle']}")
    print(f"📊 Confidence: {conf * 100:.2f}%")
# ----------------- Dataset Consistency Checker -----------------
# Collect actual class names from dataset
actual_classes = list(train_gen.class_indices.keys())
fabric_info_classes = list(FABRIC_INFO.keys())

# Check 1: Classes in dataset but missing in FABRIC_INFO
missing_in_info = [c for c in actual_classes if c not in fabric_info_classes]

# Check 2: Keys in FABRIC_INFO not present in dataset
missing_in_dataset = [c for c in fabric_info_classes if c not in actual_classes]

print("\n===== DATASET CONSISTENCY CHECK =====")
print("Classes found in dataset:", actual_classes)
if missing_in_info:
    print("⚠️ Missing in FABRIC_INFO:", missing_in_info)
else:
    print("✅ All dataset classes found in FABRIC_INFO.")

if missing_in_dataset:
    print("ℹ️ Extra keys in FABRIC_INFO not present in dataset:", missing_in_dataset)
else:
    print("✅ All FABRIC_INFO keys correspond to dataset classes.")
print("====================================\n")


# Function for live prediction via webcam
def live_predict(model):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not access webcam.")
        return

    print("Press 'c' to capture and predict, 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Webcam', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, IMG_SIZE)
            arr = np.expand_dims(img / 255.0, axis=0)
            pred = model.predict(arr)
            idx = np.argmax(pred)
            label = label_map[idx]
            info = FABRIC_INFO.get(label, {"mix": "-", "bio": "-", "time": "-", "recycle": "-"})
            print(f"\nPrediction: {label} | Mix: {info['mix']} | Bio: {info['bio']} | Time: {info['time']} | Recycle: {info['recycle']}")
        elif key == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()